In [2]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=dd73d176b1b4333075458ede090806db6ddf057837ae1be4b9f47923f2571e47
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from pyspark.sql import SparkSession
from pathlib import Path

In [ ]:
from google.colab import files

uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

In [1]:
spark = SparkSession.builder \
    .appName("Read CSV with PySpark") \
    .getOrCreate()

NameError: name 'SparkSession' is not defined

In [ ]:
csv_file_path = "realtor-data.zip.csv"

In [ ]:
df = spark.read.csv(str(csv_file_path), header=True, inferSchema=True)
df.show()

+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|prev_sold_date|
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|     Adjuntas|Puerto Rico|     601|     920.0|          NULL|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|     Adjuntas|Puerto Rico|     601|    1527.0|          NULL|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|   Juana Diaz|Puerto Rico|     795|     748.0|          NULL|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|        Ponce|Puerto Rico|     731|    1800.0|          NULL|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|     Mayaguez|Puerto Rico|     680|      NULL|          NULL|
|   103378.0|for_sale|179000.0| 

In [ ]:
df = df.drop("prev_sold_date")
df.show()

+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|     Adjuntas|Puerto Rico|     601|     920.0|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|     Adjuntas|Puerto Rico|     601|    1527.0|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|   Juana Diaz|Puerto Rico|     795|     748.0|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|        Ponce|Puerto Rico|     731|    1800.0|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|     Mayaguez|Puerto Rico|     680|      NULL|
|   103378.0|for_sale|179000.0|  4|   3|    0.46|1850806.0|San Sebastian|Puerto Rico|     612|    2520.0|
|     1205.0|for_sale| 50000.0|  3|   1|     0

In [ ]:
df.count()

2226382

In [ ]:
df_clean = df.dropna()
df_clean.count()

1354105

In [ ]:
df_clean = df_clean.dropDuplicates()
df_clean.count()

1353936

In [ ]:
df_clean.createOrReplaceTempView("data")

In [ ]:
michigan_homes = spark.sql("""
  SELECT *
  FROM data
  WHERE state = 'Michigan'
  """)
michigan_homes.show()

+-----------+--------+--------+---+----+--------+---------+--------------------+--------+--------+----------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|                city|   state|zip_code|house_size|
+-----------+--------+--------+---+----+--------+---------+--------------------+--------+--------+----------+
|    63967.0|for_sale|  9900.0|  2|   1|    0.13| 798491.0|             Inkster|Michigan|   48141|     808.0|
|    98474.0|for_sale|483500.0|  4|   4|     5.0|1538869.0|         Ottawa Lake|Michigan|   49267|    3565.0|
|    75202.0|for_sale|125000.0|  3|   2|    0.14|  81756.0|            Westland|Michigan|   48186|    1135.0|
|    13061.0|for_sale|109900.0|  3|   1|    0.09| 314326.0|             Detroit|Michigan|   48223|    1228.0|
|    35524.0|for_sale|169900.0|  3|   3|    0.41| 545314.0|               Niles|Michigan|   49120|    2100.0|
|    79460.0|for_sale|339900.0|  3|   3|    0.48|1093874.0|            Bridgman|Michigan|   49106|    2196.0|
|    56189

In [ ]:
michigan_homes.toPandas().to_csv('michigan_homes.csv', index=False)


In [ ]:
michigan_homes.printSchema()

root
 |-- brokered_by: double (nullable = true)
 |-- status: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bed: integer (nullable = true)
 |-- bath: integer (nullable = true)
 |-- acre_lot: double (nullable = true)
 |-- street: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- house_size: double (nullable = true)



In [ ]:
# spark.stop()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
df_michigan = michigan_homes.drop('city','state','status')

In [ ]:
# df_michigan = df_michigan.drop('city','status_index','brokered_by')

In [ ]:
# filtered_df = df_michigan.filter(df_michigan['price']>=75000)

In [ ]:
feature_cols = ['bed', 'bath', 'acre_lot','zip_code', 'house_size','brokered_by'
                  ]



In [ ]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df_michigan)

In [ ]:
# categorical = ['status','city','state','zip_code','street','brokered_by']
# numerical = ['bed','bath','acre_lot','house_size']

In [ ]:
rf_df_michigan = df.select("features", "price")



In [ ]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
rf = RandomForestRegressor(labelCol="price", featuresCol="features", numTrees=100)
rf_model = rf.fit(train_data)

In [ ]:
predictions = rf_model.transform(test_data)


In [ ]:
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(f"r2): {r2}")

r2): 0.542698135018403


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
importances = rf_model.featureImportances



In [ ]:
df_michigan.show()

+-----------+--------+---+----+--------+---------+--------+----------+
|brokered_by|   price|bed|bath|acre_lot|   street|zip_code|house_size|
+-----------+--------+---+----+--------+---------+--------+----------+
|    63967.0|  9900.0|  2|   1|    0.13| 798491.0|   48141|     808.0|
|    98474.0|483500.0|  4|   4|     5.0|1538869.0|   49267|    3565.0|
|    75202.0|125000.0|  3|   2|    0.14|  81756.0|   48186|    1135.0|
|    13061.0|109900.0|  3|   1|    0.09| 314326.0|   48223|    1228.0|
|    35524.0|169900.0|  3|   3|    0.41| 545314.0|   49120|    2100.0|
|    79460.0|339900.0|  3|   3|    0.48|1093874.0|   49106|    2196.0|
|    56189.0|650000.0|  3|   2|    0.65| 435092.0|   49129|    1705.0|
|    74532.0|184900.0|  2|   2|    1.44|1541394.0|   49111|    2165.0|
|    22025.0|199900.0|  4|   4|     0.2|1203957.0|   49120|    3715.0|
|    23545.0|175000.0|  4|   1|    0.13|1068301.0|   48030|    1312.0|
|    81326.0|240900.0|  3|   1|     0.6|1221515.0|   48038|    1057.0|
|    3

In [ ]:
# Evaluate the model
from sklearn.metrics import mean_absolute_error

# Predict on test set
y_pred = model.predict(X_test)

# Calculate mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

In [ ]:
spark.stop()